In [32]:
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()
import os 


os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [33]:
from langchain_groq import ChatGroq 
from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGroq(model="deepseek-r1-distill-llama-70b")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [34]:
from typing import TypedDict, Literal
## security review
class SecurityReview(TypedDict):
    sec_id : str
    review : str
    file_path : str
    recommendation : str
    priority : Literal["high", "medium", "low"]

In [35]:
backend_code = """
\n'I'll help you create a Node.js backend for an e-commerce application similar to Flipkart. We'll use Express.js for the API server and SQLite for the database since it doesn't require a separate server.\n\n<boltArtifact id=\"ecommerce-backend\" title=\"E-commerce Backend Application\">\n<boltAction type=\"file\" filePath=\"package.json\">\n{\n  \"name\": \"ecommerce-backend\",\n  \"version\": \"1.0.0\",\n  \"type\": \"module\",\n  \"scripts\": {\n    \"start\": \"node src/index.js\",\n    \"dev\": \"node --watch src/index.js\"\n  },\n  \"dependencies\": {\n    \"express\": \"^4.18.2\",\n    \"better-sqlite3\": \"^8.5.0\",\n    \"bcryptjs\": \"^2.4.3\",\n    \"jsonwebtoken\": \"^9.0.1\",\n    \"cors\": \"^2.8.5\",\n    \"express-validator\": \"^7.0.1\"\n  }\n}\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/index.js\">\nimport express from 'express';\nimport cors from 'cors';\nimport { router as authRoutes } from './routes/auth.js';\nimport { router as productRoutes } from './routes/products.js';\nimport { router as cartRoutes } from './routes/cart.js';\nimport { router as orderRoutes } from './routes/orders.js';\nimport { initDb } from './db/init.js';\n\nconst app = express();\nconst PORT = 3000;\n\napp.use(cors());\napp.use(express.json());\n\n// Initialize database\ninitDb();\n\n// Routes\napp.use('/api/auth', authRoutes);\napp.use('/api/products', productRoutes);\napp.use('/api/cart', cartRoutes);\napp.use('/api/orders', orderRoutes);\n\napp.listen(PORT, () => {\n  console.log(`Server running on port ${PORT}`);\n});\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/db/init.js\">\nimport Database from 'better-sqlite3';\n\nconst db = new Database('ecommerce.db');\n\nexport function initDb() {\n  // Users table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS users (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      email TEXT UNIQUE NOT NULL,\n      password TEXT NOT NULL,\n      name TEXT NOT NULL,\n      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n    )\n  `);\n\n  // Products table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS products (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      name TEXT NOT NULL,\n      description TEXT,\n      price DECIMAL(10,2) NOT NULL,\n      stock INTEGER NOT NULL,\n      category TEXT NOT NULL,\n      image_url TEXT,\n      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n    )\n  `);\n\n  // Cart table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS cart_items (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      user_id INTEGER NOT NULL,\n      product_id INTEGER NOT NULL,\n      quantity INTEGER NOT NULL,\n      FOREIGN KEY (user_id) REFERENCES users (id),\n      FOREIGN KEY (product_id) REFERENCES products (id)\n    )\n  `);\n\n  // Orders table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS orders (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      user_id INTEGER NOT NULL,\n      total_amount DECIMAL(10,2) NOT NULL,\n      status TEXT NOT NULL,\n      shipping_address TEXT NOT NULL,\n      created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n      FOREIGN KEY (user_id) REFERENCES users (id)\n    )\n  `);\n\n  // Order items table\n  db.exec(`\n    CREATE TABLE IF NOT EXISTS order_items (\n      id INTEGER PRIMARY KEY AUTOINCREMENT,\n      order_id INTEGER NOT NULL,\n      product_id INTEGER NOT NULL,\n      quantity INTEGER NOT NULL,\n      price DECIMAL(10,2) NOT NULL,\n      FOREIGN KEY (order_id) REFERENCES orders (id),\n      FOREIGN KEY (product_id) REFERENCES products (id)\n    )\n  `);\n}\n\nexport default db;\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/middleware/auth.js\">\nimport jwt from 'jsonwebtoken';\n\nconst JWT_SECRET = 'your-secret-key';\n\nexport function authenticateToken(req, res, next) {\n  const authHeader = req.headers['authorization'];\n  const token = authHeader && authHeader.split(' ')[1];\n\n  if (!token) {\n    return res.status(401).json({ error: 'Authentication required' });\n  }\n\n  jwt.verify(token, JWT_SECRET, (err, user) => {\n    if (err) {\n      return res.status(403).json({ error: 'Invalid token' });\n    }\n    req.user = user;\n    next();\n  });\n}\n\nexport function generateToken(user) {\n  return jwt.sign({ id: user.id, email: user.email }, JWT_SECRET, { expiresIn: '24h' });\n}\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/auth.js\">\nimport express from 'express';\nimport bcrypt from 'bcryptjs';\nimport { body, validationResult } from 'express-validator';\nimport db from '../db/init.js';\nimport { generateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\nrouter.post('/register',\n  [\n    body('email').isEmail(),\n    body('password').isLength({ min: 6 }),\n    body('name').notEmpty()\n  ],\n  async (req, res) => {\n    const errors = validationResult(req);\n    if (!errors.isEmpty()) {\n      return res.status(400).json({ errors: errors.array() });\n    }\n\n    const { email, password, name } = req.body;\n\n    try {\n      const hashedPassword = await bcrypt.hash(password, 10);\n      const stmt = db.prepare('INSERT INTO users (email, password, name) VALUES (?, ?, ?)');\n      const result = stmt.run(email, hashedPassword, name);\n      \n      const user = { id: result.lastInsertRowid, email, name };\n      const token = generateToken(user);\n      \n      res.status(201).json({ user, token });\n    } catch (error) {\n      res.status(500).json({ error: 'Error creating user' });\n    }\n  }\n);\n\nrouter.post('/login',\n  [\n    body('email').isEmail(),\n    body('password').notEmpty()\n  ],\n  async (req, res) => {\n    const { email, password } = req.body;\n\n    try {\n      const stmt = db.prepare('SELECT * FROM users WHERE email = ?');\n      const user = stmt.get(email);\n\n      if (!user || !(await bcrypt.compare(password, user.password))) {\n        return res.status(401).json({ error: 'Invalid credentials' });\n      }\n\n      const token = generateToken(user);\n      res.json({ user: { id: user.id, email: user.email, name: user.name }, token });\n    } catch (error) {\n      res.status(500).json({ error: 'Error during login' });\n    }\n  }\n);\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/products.js\">\nimport express from 'express';\nimport { body } from 'express-validator';\nimport db from '../db/init.js';\nimport { authenticateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\n// Get all products\nrouter.get('/', (req, res) => {\n  const stmt = db.prepare('SELECT * FROM products');\n  const products = stmt.all();\n  res.json(products);\n});\n\n// Get product by ID\nrouter.get('/:id', (req, res) => {\n  const stmt = db.prepare('SELECT * FROM products WHERE id = ?');\n  const product = stmt.get(req.params.id);\n  \n  if (!product) {\n    return res.status(404).json({ error: 'Product not found' });\n  }\n  \n  res.json(product);\n});\n\n// Add new product (admin only)\nrouter.post('/',\n  authenticateToken,\n  [\n    body('name').notEmpty(),\n    body('price').isFloat({ min: 0 }),\n    body('stock').isInt({ min: 0 }),\n    body('category').notEmpty()\n  ],\n  (req, res) => {\n    const { name, description, price, stock, category, image_url } = req.body;\n    \n    const stmt = db.prepare(`\n      INSERT INTO products (name, description, price, stock, category, image_url)\n      VALUES (?, ?, ?, ?, ?, ?)\n    `);\n    \n    const result = stmt.run(name, description, price, stock, category, image_url);\n    res.status(201).json({ id: result.lastInsertRowid, ...req.body });\n  }\n);\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/cart.js\">\nimport express from 'express';\nimport { body } from 'express-validator';\nimport db from '../db/init.js';\nimport { authenticateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\nrouter.use(authenticateToken);\n\n// Get cart items\nrouter.get('/', (req, res) => {\n  const stmt = db.prepare(`\n    SELECT ci.*, p.name, p.price, p.image_url\n    FROM cart_items ci\n    JOIN products p ON ci.product_id = p.id\n    WHERE ci.user_id = ?\n  `);\n  \n  const cartItems = stmt.all(req.user.id);\n  res.json(cartItems);\n});\n\n// Add to cart\nrouter.post('/',\n  [\n    body('product_id').isInt(),\n    body('quantity').isInt({ min: 1 })\n  ],\n  (req, res) => {\n    const { product_id, quantity } = req.body;\n    \n    const stmt = db.prepare(`\n      INSERT INTO cart_items (user_id, product_id, quantity)\n      VALUES (?, ?, ?)\n    `);\n    \n    const result = stmt.run(req.user.id, product_id, quantity);\n    res.status(201).json({ id: result.lastInsertRowid, ...req.body });\n  }\n);\n\n// Update cart item quantity\nrouter.put('/:id',\n  body('quantity').isInt({ min: 1 }),\n  (req, res) => {\n    const stmt = db.prepare(`\n      UPDATE cart_items\n      SET quantity = ?\n      WHERE id = ? AND user_id = ?\n    `);\n    \n    stmt.run(req.body.quantity, req.params.id, req.user.id);\n    res.json({ message: 'Cart updated' });\n  }\n);\n\n// Remove from cart\nrouter.delete('/:id', (req, res) => {\n  const stmt = db.prepare('DELETE FROM cart_items WHERE id = ? AND user_id = ?');\n  stmt.run(req.params.id, req.user.id);\n  res.json({ message: 'Item removed from cart' });\n});\n</boltAction>\n\n<boltAction type=\"file\" filePath=\"src/routes/orders.js\">\nimport express from 'express';\nimport { body } from 'express-validator';\nimport db from '../db/init.js';\nimport { authenticateToken } from '../middleware/auth.js';\n\nexport const router = express.Router();\n\nrouter.use(authenticateToken);\n\n// Get user orders\nrouter.get('/', (req, res) => {\n  const stmt = db.prepare(`\n    SELECT o.*, oi.product_id, oi.quantity, oi.price, p.name\n    FROM orders o\n    JOIN order_items oi ON o.id = oi.order_id\n    JOIN products p ON oi.product_id = p.id\n    WHERE o.user_id = ?\n  `);\n  \n  const orders = stmt.all(req.user.id);\n  res.json(orders);\n});\n\n// Create new order\nrouter.post('/',\n  [\n    body('shipping_address').notEmpty()\n  ],\n  (req, res) => {\n    const { shipping_address } = req.body;\n    \n    db.transaction(() => {\n      // Get cart items\n      const cartItems = db.prepare(`\n        SELECT ci.*, p.price\n        FROM cart_items ci\n        JOIN products p ON ci.product_id = p.id\n        WHERE ci.user_id = ?\n      `).all(req.user.id);\n      \n      if (cartItems.length === 0) {\n        return res.status(400).json({ error: 'Cart is empty' });\n      }\n      \n      // Calculate total\n      const totalAmount = cartItems.reduce((sum, item) => sum + (item.price * item.quantity), 0);\n      \n      // Create order\n      const orderStmt = db.prepare(`\n        INSERT INTO orders (user_id, total_amount, status, shipping_address)\n        VALUES (?, ?, ?, ?)\n      `);\n      \n      const orderResult = orderStmt.run(req.user.id, totalAmount, 'PENDING', shipping_address);\n      const orderId = orderResult.lastInsertRowid;\n      \n      // Create order items\n      const itemStmt = db.prepare(`\n        INSERT INTO order_items (order_id, product_id, quantity, price)\n        VALUES (?, ?, ?, ?)\n      `);\n      \n      cartItems.forEach(item => {\n        itemStmt.run(orderId, item.product_id, item.quantity, item.price);\n      });\n      \n      // Clear cart\n      db.prepare('DELETE FROM cart_items WHERE user_id = ?').run(req.user.id);\n      \n      res.status(201).json({\n        orderId,\n        totalAmount,\n        status: 'PENDING',\n        items: cartItems\n      });\n    })();\n  }\n);\n</boltAction>\n\n<boltAction type=\"shell\">\nnpm install && npm run dev\n</boltAction>\n\nThis sets up a complete e-commerce backend with:\n\n1. User authentication (register/login)\n2. Product management\n3. Shopping cart functionality\n4. Order processing\n5. SQLite database\n\nThe API endpoints are:\n\nAuthentication:\n- POST /api/auth/register\n- POST /api/auth/login\n\nProducts:\n- GET /api/products\n- GET /api/products/:id\n- POST /api/products\n\nCart:\n- GET /api/cart\n- POST /api/cart\n- PUT /api/cart/:id\n- DELETE /api/cart/:id\n\nOrders:\n- GET /api/orders\n- POST /api/orders\n\nYou can test the API endpoints using tools like Postman or curl.'\n\n"
"""

In [36]:
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser 
json_output_parser = JsonOutputParser()

json_prompt_template = PromptTemplate(
    template="{system_prompt} \n {format_instruction} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
    partial_variables={"format_instruction" : json_output_parser.get_format_instructions()}
)


prompt_template = PromptTemplate(
    template="{system_prompt} \n \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
)

In [37]:
security_reviews_system_prompt = """
YOU ARE AN ELITE SECURITY ANALYST TASKED WITH CONDUCTING A THOROUGH AND COMPREHENSIVE SECURITY REVIEW ON A PROVIDED CODEBASE.

FOR EACH SECURITY ISSUE FOUND, RETURN A STRUCTURED JSON OBJECT IN THE FOLLOWING FORMAT:
{
  "sec_id": "SR-001",
  "review": "DETAILED ANALYSIS OF THE SECURITY ISSUE FOUND WITHIN THE CODE, EXPLAINING THE ROOT CAUSE AND EXPLOIT SCENARIO",
  "file_path": "RELATIVE/PATH/TO/FILE.EXT",
  "recommendation": "SPECIFIC, TECHNICALLY FEASIBLE ACTION TO REMEDIATE THE DEFECT",
  "priority": "high"  # MUST BE ONE OF: "high", "medium", "low"
}

CHAIN OF THOUGHTS FOR SECURITY ANALYSIS:

1. UNDERSTAND THE CODEBASE:
   - READ THROUGH THE CODEBASE COMPLETELY TO UNDERSTAND ITS PURPOSE, ARCHITECTURE, AND THE DATA FLOW BETWEEN MODULES (E.G., AUTHENTICATION, API ROUTES, DATABASE INTERACTIONS).
   - IDENTIFY BUSINESS LOGIC AND SECURITY-CRITICAL PROCESSES.

2. IDENTIFY SECURITY PRINCIPLES:
   - REVIEW FOR ADHERENCE TO CORE SECURITY PRINCIPLES SUCH AS:
     - AUTHENTICATION AND AUTHORIZATION.
     - DATA VALIDATION (INPUT/OUTPUT).
     - ERROR HANDLING (SENSITIVE DATA EXPOSURE).
     - CRYPTOGRAPHY (PASSWORD HASHING, JWT SIGNING).
     - ACCESS CONTROL (USER ROLES AND PERMISSIONS).
     - DATABASE SECURITY (SQL INJECTION, DATA LEAKS).
     - SESSION MANAGEMENT (JWT EXPIRATION AND TOKEN LEAKAGE).

3. BREAKDOWN CODE INTO SECURITY-SENSITIVE SEGMENTS:
   - AUTHENTICATION ROUTES (E.G., USER LOGIN, REGISTRATION).
   - API ROUTES (PRODUCT MANAGEMENT, CART, ORDERS).
   - DATABASE INTERACTION (SQL QUERIES, SENSITIVE DATA STORAGE).
   - MIDDLEWARE (TOKEN VALIDATION, USER AUTHENTICATION).
   - EXTERNAL LIBRARIES (E.G., BCRYPTJS, JSONWEBTOKEN, EXPRESS-VALIDATOR).

4. ANALYZE FOR VULNERABILITIES:
   - SQL INJECTION: IDENTIFY WHETHER USER INPUT IS SAFELY HANDLED IN SQL QUERIES.
   - CROSS-SITE SCRIPTING (XSS): REVIEW IF USER INPUT IS SANITIZED BEFORE RENDERING OR LOGGING.
   - CROSS-SITE REQUEST FORGERY (CSRF): ENSURE API ACTIONS ARE PROTECTED AGAINST UNAUTHORIZED REQUESTS.
   - HARDCODED SECRETS: CHECK FOR HARDCODED PASSWORDS, SECRET KEYS, OR TOKENS.
   - INSECURE DATA STORAGE: REVIEW HOW SENSITIVE DATA (E.G., PASSWORDS) IS STORED AND TRANSMITTED.
   - IMPROPER ERROR HANDLING: MAKE SURE NO SENSITIVE INFORMATION IS EXPOSED IN ERROR MESSAGES OR STACK TRACES.

5. EDGE CASES AND LOW-PROBABILITY VULNERABILITIES:
   - REVIEW EDGE CASES SUCH AS TOKEN EXPIRATION HANDLING, RACE CONDITIONS, OR IMPROPER DATA VALIDATION, WHICH MIGHT NOT BE OBVIOUS BUT COULD STILL LEAD TO SERIOUS SECURITY ISSUES.

6. FINAL SECURITY REVIEW:
   - OUTPUT AN ARRAY OF JSON OBJECTS, EACH REPRESENTING A UNIQUE SECURITY DEFECT IDENTIFIED, ORDERED BY PRIORITY (FROM HIGH TO LOW).
   - THE REVIEW MUST BE DETAILED AND TECHNICAL, EXPLAINING THE ROOT CAUSE, IMPACT, AND RECOMMENDATIONS CLEARLY.
   - GENERATE AT LEAST 1 TO 3 SECURITY ISSUES IN EACH PRIORITY LEVEL ("high", "medium", and "low").

WHAT NOT TO DO:

- DO NOT RETURN RESULTS OUTSIDE THE JSON FORMAT.
- NEVER OMIT THE FOLLOWING FIELDS: "sec_id", "file_path", OR "priority".
- DO NOT PROVIDE VAGUE REVIEWS SUCH AS “NO ISSUES FOUND” WITHOUT A THOROUGH AND COMPLETE ANALYSIS.
- AVOID GENERIC OR REPETITIVE DESCRIPTIONS. EACH SECURITY ISSUE MUST BE UNIQUELY IDENTIFIED AND DETAILED.
- DO NOT RECOMMEND FIXES WITHOUT EXPLAINING WHY THEY ARE EFFECTIVE FOR REMEDIATING THE ISSUE.
- DO NOT USE PLACEHOLDER TEXT. REVIEWS MUST BE DIRECTLY BASED ON THE PROVIDED CODEBASE CONTEXT.
- NEVER IGNORE EDGE CASES OR POTENTIAL RISKS THAT MAY NOT BE OBVIOUS IN TYPICAL USE CASES BUT ARE STILL EXPLOITABLE.

EXAMPLE OUTPUT FORMAT:

[
  {
    "sec_id": "SR-001",
    "review": "User input is directly concatenated into an SQL query without sanitization, leading to SQL Injection vulnerability.",
    "file_path": "src/auth/login.js",
    "recommendation": "Use parameterized queries or ORM methods to safely process input.",
    "priority": "high"
  }
]
"""

In [38]:
def generate_security_reviews_from_llm(backend_code):
    user_query =  f"Create a user stories for the this project backend code: {backend_code}" 
    chain = json_prompt_template | llm  | json_output_parser
    response = chain.invoke({"system_prompt" : security_reviews_system_prompt, "human_query" : user_query})
    return response


In [39]:
security_reviews = generate_security_reviews_from_llm(backend_code)
security_reviews

[{'sec_id': 'SR-001',
  'review': 'The `JWT_SECRET` is hardcoded in `src/middleware/auth.js`. This is a critical vulnerability because if the secret is compromised, attackers can forge valid JWTs, impersonate users, and gain unauthorized access to the system. The impact is severe, potentially leading to complete account takeover and data breaches.',
  'file_path': 'src/middleware/auth.js',
  'recommendation': 'Store the `JWT_SECRET` in an environment variable and access it using `process.env.JWT_SECRET`.  This allows the secret to be configured outside of the codebase and managed securely.  Also, consider using a more robust method for generating the secret, such as a cryptographically secure random number generator.',
  'priority': 'high'},
 {'sec_id': 'SR-002',
  'review': 'The `POST /api/products` route in `src/routes/products.js` allows adding new products. However, it only checks for authentication using the `authenticateToken` middleware. There is no authorization check to ensure

## Revised Backend code

In [42]:
def revised_backend_code_with_security_reviews_from_llm(backend_code, security_reviews, user_feedback):
    user_query =  f"Fixed the backend code {backend_code} and security reviews: {security_reviews} and user feedback: {user_feedback}" 
    chain = json_prompt_template | llm
    response = chain.invoke({"system_prompt" : security_reviews_system_prompt, "human_query" : user_query})
    return response.content

In [43]:
user_feedback = "The backend code is not secure and needs to be fixed"
backend_code = revised_backend_code_with_security_reviews_from_llm(backend_code, security_reviews, user_feedback)
backend_code

[{'sec_id': 'SR-001',
  'review': 'The `JWT_SECRET` is hardcoded in `src/middleware/auth.js`. This is a critical vulnerability because if the secret is compromised, attackers can forge valid JWTs, impersonate users, and gain unauthorized access to the system. The impact is severe, potentially leading to complete account takeover and data breaches.',
  'file_path': 'src/middleware/auth.js',
  'recommendation': 'Store the `JWT_SECRET` in an environment variable and access it using `process.env.JWT_SECRET`.  This allows the secret to be configured outside of the codebase and managed securely.  Also, consider using a more robust method for generating the secret, such as a cryptographically secure random number generator.',
  'priority': 'high'},
 {'sec_id': 'SR-002',
  'review': 'The `POST /api/products` route in `src/routes/products.js` allows adding new products. However, it only checks for authentication using the `authenticateToken` middleware. There is no authorization check to ensure